In [53]:
import numpy as np
import numpy
import argparse
import os
import cv2

import torch
import torch.nn as nn
import torchvision
import torch.utils.data
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

from PIL import Image
from engine import train_one_epoch, evaluate
import pandas as pd
from tqdm import tqdm

import matplotlib
matplotlib.use('TkAgg')
%matplotlib inline
import matplotlib.pyplot as plt

import utils
import transforms as T
import time

from scipy import ndimage
from skimage.feature import corner_harris, corner_peaks, corner_subpix
from skimage.feature import peak_local_max

In [54]:
def get_transform(train):
    transforms = []
    transforms.append(T.ToTensor())
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

In [55]:
def get_model_instance_segmentation(num_classes):
    # load an instance segmentation model pre-trained pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)
    model.backbone.body.conv1 = nn.Conv2d(36, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model

In [56]:
class StarDataset(object):
    def __init__(self, path, transforms, window_size):
        self.root = path
        self.transforms = transforms
        pth = os.listdir(path)
        self.label_sequences = []

        self.dir_paths = []
        if os.path.isdir(path):
            self.dir_paths.append(path + '/')
            print(f"Loaded {path}")

        self.window_size = window_size
        self.seq_indexs = []
        self.seq_indexs.append((0, 0, len(pth)))

    def __len__(self):
        return self.seq_indexs[-1][-1]

    def __getitem__(self, idx):
        # load images and masks
        for i, start, end in self.seq_indexs:
            if idx >= start and idx < end:
                real_idx = idx - start

                if real_idx > 3:
                    entire_data1 = np.load(self.dir_paths[i] + '/' + str(int(real_idx) - 3) + ".npy", allow_pickle=True)
                    entire_data2 = np.load(self.dir_paths[i] + '/' + str(int(real_idx) - 2) + ".npy", allow_pickle=True)
                    entire_data3 = np.load(self.dir_paths[i] + '/' + str(int(real_idx) - 1) + ".npy", allow_pickle=True)
                    entire_data4 = np.load(self.dir_paths[i] + '/' + str(int(real_idx) + 0) + ".npy", allow_pickle=True)
                else:
                    entire_data1 = np.load(self.dir_paths[i] + '/' + str(int(real_idx)) + ".npy", allow_pickle=True)
                    entire_data2 = entire_data1
                    entire_data3 = entire_data1
                    entire_data4 = entire_data1

                data1 = entire_data1[0]
                data2 = entire_data2[0]
                data3 = entire_data3[0]
                data4 = entire_data4[0]
                
                masks = entire_data4[1]

                break

        input_data = self.preprocessing(data1, data2, data3, data4)
        return input_data, torch.as_tensor(np.array(masks), dtype=torch.uint8)

    def preprocessing(self, data1, data2, data3, data4):
        # 0 ground 1 air 2 building 3 spell 4 ground 5 air 6 building 7 spell 8 resource 9 vision 10 terrain
        temp = np.zeros([self.window_size, 9, data1.shape[1], data1.shape[2]])

        temp[0] = data1
        temp[1] = data2
        temp[2] = data3
        temp[3] = data4

        data = temp
        data = data.reshape(self.window_size * data.shape[1], data.shape[2], -1)
        # #data = data.reshape(self.window_size*data.shape[0],data.shape[1],-1)
        # label = np.array([label[0]/3456, label[1]/3720])
        return torch.FloatTensor(data)

In [ ]:
from glob import glob
import os
import pandas as pd
import numpy as np

def load(path: str, replay: int):
    vpd_paths = glob(os.path.join(path, "*/") + "{}.rep.vpd".format(str(replay)))
    print(vpd_paths)
    vpds = [pd.read_csv(_, index_col=None) for _ in vpd_paths]
    return vpds

replay = 212
datset_dir =  "../data/vpds2/"
a = load(datset_dir,replay)


In [ ]:
vpx_array = a[0]['vpx'].to_numpy() # bcm
vpy_array = a[0]['vpy'].to_numpy() # bc,

In [ ]:
testing_replay = ["212"]
datset_dir = "C:/Users/joo/PycharmProjects/starcraft_replay_reconstructor-devel/starcraft_replay_reconstructor-devel/result/"

for i in testing_replay:    
    replay_dir = datset_dir + str(i)
    replay_name = int(replay_dir.split('/')[-1])
    print(replay_name)
    dataset = StarDataset(replay_dir, get_transform(train=False), window_size=4)
    
    dataset_len = len(dataset)    
    Start, End, Step = 0, len(dataset), 1
    test_img_array = []
    test_img_one_channel_array = []
    test_target_array = []

    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    for i in range(Start, End, Step):
        img_t = dataset[i][0]
        target_t = dataset[i][1]
        test_img_array.append(img_t)
        img_one_channel = img_t.sum(axis=0, keepdim=True)
        test_img_one_channel_array.append(img_one_channel)
        test_target_array.append(target_t)


In [ ]:
%matplotlib inline

fig, ax = plt.subplots(1,1)
acac1 = test_img_array[3000][:2].sum(axis=0).detach().numpy()    
acac1 = np.where(acac1 >= 1, 0.16, 0.0)
acac2 = test_img_array[3000][2:3].sum(axis=0).detach().numpy()    
acac2 = np.where(acac2 >= 1, 0.32, 0.0)
acac3 = test_img_array[3000][3:4].sum(axis=0).detach().numpy()    
acac3 = np.where(acac2 >= 1, 0.48, 0.0)

acac4 = test_img_array[3000][4:6].sum(axis=0).detach().numpy()    
acac4 = np.where(acac4 >= 1, 0.64, 0.0)
acac5 = test_img_array[3000][6:7].sum(axis=0).detach().numpy()    
acac5 = np.where(acac5 >= 1, 0.8, 0.0)
acac6 = test_img_array[3000][7:8].sum(axis=0).detach().numpy()    
acac6 = np.where(acac6 >= 1, 0.96, 0.0)

viewport_x = int(vpx_array[3000]/32)
viewport_y = int(vpy_array[3000]/32)

acac = acac1 + acac2 + acac3  + acac4  + acac5  + acac6
acac = np.where(acac >= 1, 0.96, acac)

acac[viewport_y:viewport_y+1, viewport_x:viewport_x+20]=1
acac[viewport_y+12:viewport_y+13, viewport_x:viewport_x+21]=1    
acac[viewport_y:viewport_y+12, viewport_x:viewport_x+1]=1
acac[viewport_y:viewport_y+12, viewport_x+20:viewport_x+21]=1
ax.imshow(acac, cmap="RdYlBu_r") # RdYlBu_r # gist_earth_r

plt.show() 

## Rule-Based 

In [ ]:
import math
import random
import gym
from gym import spaces
import pandas as pd
import numpy as np
from skimage.metrics import mean_squared_error
from skimage.metrics import structural_similarity as ssim
action = 0

class StarObserverEnv2(gym.Env):
    def __init__(self, render_mode=None):
        super(StarObserverEnv2, self).__init__()        
        
        #self.reward_range = (0, MAX_ACCOUNT_BALANCE)
        N_DISCRETE_ACTIONS = 9        
        N_CHANNELS, HEIGHT, WIDTH = 8, 96, 20
        
        self.action_space = spaces.Discrete(N_DISCRETE_ACTIONS)
        self.current_step = 0
        self.current_episode_step = 0
        self.current_action = 0
        # self.previous_state = torch.zeros(16, 20, 12)
        self.render_mode = render_mode
        
        self.observation_space = spaces.Box(low=0, high=1, shape=
                    (N_CHANNELS, HEIGHT, WIDTH), dtype=np.float16)
        self.total_angle = []
        self.action_ = ["left,up", "up", "up,right", "left", "stop", "right", "down,left", "down", "right,down"]   
        self.action_dic = {0: ['-','-'], 1: ['-','0'],2: ['-','+'],3: ['0','-'],4: ['0', '0'],
            5: ['0','+'],6: ['+','-'],7: ['+','0'],8: ['+','+']}
        self.pre_frame_center_coor = 0,0
        self.curr_frame_center_coor = 0,0
        
    def _next_observation(self):
        self.current_step

#         viewport_x = int(predictions[self.current_step][0]['boxes'][0][0])
#         viewport_y = int(predictions[self.current_step][0]['boxes'][0][1])        
#         print("vx",viewport_x)
#         print(viewport_y)
        
        
        viewport_x =  21
        viewport_y =  25
        
        if viewport_x == 0 or viewport_y == 0 or viewport_x == (128-20) or viewport_y == (128-12) :
            done = 1
            return 0,0, done

        state1 = []
        next_state1 = []
        
        for i in range(4):        
            current_state_channel = test_img_array[self.current_step-3+i][0:8]
            current_state_viewport = current_state_channel.clone()[:,viewport_y:viewport_y+12, viewport_x:viewport_x+20]
            
            next_state_channel = test_img_array[self.current_step+i+1][0:8]
            next_state_viewport = next_state_channel.clone()[:,viewport_y:viewport_y+12, viewport_x:viewport_x+20]
            
            state1.append(current_state_viewport)
            next_state1.append(next_state_viewport)
                
        current_state = torch.cat([torch.cat(state1,dim=1), torch.cat(next_state1,dim=1)],dim=1) # torch.Size([8, 48, 20]), [8, 48, 20]
        # current_state, torch.Size([8, 96, 20])
                
        ##########################################################################
        # 0: left+up   # 1: up         # 2: up+right   #3: left              #4: stop
        # 5: right    # 6: down+left   # 7: down       # 8: right + down    

        action_list = []
        pixel_speed = 1

        if self.action_dic[action][0] == "-":            
            action_list.append(viewport_y-pixel_speed)
            action_list.append(viewport_y+12-pixel_speed)
        elif self.action_dic[action][0] == "0":
            action_list.append(viewport_y)
            action_list.append(viewport_y+12)
        elif self.action_dic[action][0] == "+":
            action_list.append(viewport_y+pixel_speed)
            action_list.append(viewport_y+12+pixel_speed)

        if self.action_dic[action][1] == "-":
            action_list.append(viewport_x-pixel_speed)
            action_list.append(viewport_x+20-pixel_speed)
        elif self.action_dic[action][1] == "0":
            action_list.append(viewport_x)
            action_list.append(viewport_x+20)
        elif self.action_dic[action][1] == "+":
            action_list.append(viewport_x+pixel_speed)
            action_list.append(viewport_x+20+pixel_speed)

        next_state2 = []
        next_next_state2 = []
        
        for i in range(4):        
            next_state_channel = test_img_array[self.current_step-3+i+4][0:8] # torch.Size([8, 128, 128])
            next_state_viewport = next_state_channel.clone()[:,action_list[0]:action_list[1], action_list[2]:action_list[3]] # torch.Size([8, 12, 20])                        
            # current_state_viewport torch.Size([8, 12, 20])
            
            next_next_state_channel = test_img_array[self.current_step+i+1+4][0:8]
            next_next_state_viewport = next_next_state_channel.clone()[:,action_list[0]:action_list[1], action_list[2]:action_list[3]]
            
            next_state2.append(next_state_viewport)
            next_next_state2.append(next_next_state_viewport)
        
        next_state = torch.cat([torch.cat(next_state2,dim=1), torch.cat(next_next_state2,dim=1)],dim=1)

        return current_state, next_state, 0

    def get_angle(self, center_coordinate1, center_coordinate2):
        y1, x1 = center_coordinate1
        y2, x2 = center_coordinate2
        degrees = math.degrees(math.atan2(y2-y1, x2-x1))
            
        if degrees < 0: # 0~ 180 ~ 0 ~ -180 -> 0 ~ 360
            degrees = 360-abs(degrees)
            
        return (720- degrees) % 360
        
    def find_center_coordinate(self, frame):
        frame = torch.sum(frame,0)
        center_cordinate =  torch.nonzero(frame).sum(0).numpy() / len(torch.nonzero(frame)) # (y,x로 찍혀나옴)
        return center_cordinate        
        
    def step(self, action):
        
        done = 0
        reward = 0
        angle_reward = 0
        distance_reward = 0
        
   
        self.current_step += 1         
        self.current_episode_step += 1
        
        state, next_state, done = self._next_observation()
        
        if self.current_episode_step == 30:
            self.current_episode_step = 0
            done = 1

        self.action_to_angle = {0:135, 1:90, 2:45, 3: 180, 4:0,
            5:0 , 6:225, 7:270, 8:315}
        
        if done == 0:            
            self.pre_frame_center_coor = self.find_center_coordinate(state[:, 24:36])
            self.curr_frame_center_coor = self.find_center_coordinate(state[:, 36:48])
            angle = self.get_angle(self.pre_frame_center_coor, self.curr_frame_center_coor)
            
#             print("pre_center_coor:",self.pre_frame_center_coor, "curr_center_coor:",self.curr_frame_center_coor)
            print("frame_angle: ", angle)            
            
            kk_minus = []
            for keys in self.action_to_angle.keys():
                kk_minus.append(abs(angle - self.action_to_angle[keys]))
            action = np.array(kk_minus).argmin()
            
            action_idx = action        
            self.current_action = action                     
            
            #  객체가 하나도 안잡히면 angle이 nan이 나옴
            if np.isnan(angle):
                angle = 0
                angle_reward -=2
                done = 1
            else :
            # 우, 우상, 상, 좌상, 좌, 좌하, 하, 우하, 정지 순임        
                if angle == 0 and action_idx == 4:
                    angle_reward += 0.5
                elif self.action_to_angle[action] <= angle+22.5 and self.action_to_angle[action] > angle-22.5:
                    angle_reward += 2
                elif self.action_to_angle[action] <= angle+45.0 and self.action_to_angle[action] > angle-45.0:
                    angle_reward += 1
        else:
            reward1 = 0
            angle_reward = 0
            distance_reward = 0
       
        reward = angle_reward   
        
        return next_state, reward, done, {}, action
        
        
    def reset(self):
        #self.current_step = random.randint(4, dataset_len - 50)        
        self.current_step = 2358 #random.randint(4, dataset_len - 50)        
        #viewport_x = int(predictions[self.current_step][0]['boxes'][0][0])
        #viewport_y = int(predictions[self.current_step][0]['boxes'][0][1])
        
        viewport_x =  21
        viewport_y =  25
        
        state = []
        next_state = []
        for i in range(4):        
            current_state_channel = test_img_array[self.current_step-i][0:8]
            current_state_viewport = current_state_channel.clone()[:,viewport_y:viewport_y+12, viewport_x:viewport_x+20]
            
            next_state_channel = test_img_array[self.current_step-i+4][0:8]
            next_state_viewport = next_state_channel.clone()[:,viewport_y:viewport_y+12, viewport_x:viewport_x+20]
            
            state.append(current_state_viewport)
            next_state.append(next_state_viewport)
        
        current_state = torch.cat([torch.cat(state,dim=1), torch.cat(next_state,dim=1)],dim=1)
        
        return current_state, self.current_step
    
    def render(self):
        x_axis, y_axis = 1,2                
        fig, ax1 = plt.subplots(x_axis, y_axis, figsize=(10, 10))
        state, next_state, done  = self._next_observation()
                
        if done == 1:
            return 0 
        else:
            pre_y, pre_x = self.pre_frame_center_coor 
            curr_y, curr_x = self.curr_frame_center_coor 
            
            
            ax1[0].imshow(state[:,24:36].sum(axis=0).detach().numpy()    , cmap="RdYlBu_r")      # 48 ~ 96 
            ax1[0].add_patch(plt.Circle((pre_x,pre_y), 0.5, color='r'))            
            ax1[0].set_title("state") #, (" + self.action_[self.current_action]+ ")")

            
            ax1[1].imshow(state[:,36:48].sum(axis=0).detach().numpy()    , cmap="RdYlBu_r")        
            ax1[1].add_patch(plt.Circle((curr_x,curr_y), 0.5, color='r'))
            ax1[1].set_title("state'")
            
            for i in ax1:
                i.set_xticks(range(0,20,4))
                i.set_yticks(range(0,12,4))
                i.grid() 

            plt.show()
            
            return 0
        

    

In [ ]:
action_sequence_list = []

In [ ]:
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import gym
from itertools import count
from gym.envs.registration import register
import matplotlib.pyplot as plt

BATCH_SIZE = 32
LR = 0.01                   # learning rate
EPSILON = 0.9               # exploration
GAMMA = 0.9                 # reward discount factor
TARGET_REPLACE_ITER = 100   # target update frequency
MEMORY_CAPACITY = 5 #2000

N_ACTIONS = 9
N_STATES = (16,12,20)

env = StarObserverEnv2()


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(8, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(23552, 1)
        self.fc2 = nn.Linear(1, 9)
        #(64x32 and 2048x32)
        #1x2048 and 32x64
        
    def forward(self, x):      
        x = self.conv1(x)      
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)      
        x = self.dropout1(x)      
        x = torch.flatten(x, 1)      
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        
        #output = F.log_softmax(x, dim=1)
        return x #output


class DQN(nn.Module):
    def __init__(self):
        super(DQN, self).__init__()
        self.eval_net = Net()
        self.target_net =  Net()
        self.train_step_counter = 0                                     # for target updating
        self.memory_counter = 0                                         # for storing memory
        self.memory_counter2 = 0      
        self.memory = np.zeros((MEMORY_CAPACITY, 8 , 96*2 , 20))     # initialize memory
        self.memory2 = np.zeros((MEMORY_CAPACITY, 2))     # initialize memory
        self.optimizer = torch.optim.Adam(self.eval_net.parameters(), lr=LR)

    def select_action(self, x):
        x = x.float().unsqueeze(0)
        if np.random.uniform() < EPSILON:   # Exploration
            actions_value = self.eval_net.forward(x) # [[0.3, 0.9]]
            action = torch.max(actions_value, 1)[1].data.numpy() # 큰 값의 인덱스를 뽑아냄 [1], (0.9 > 0.3)
            action = action[0] # 1
        else:
            action = np.random.randint(0, N_ACTIONS)
        return action # 0과 1을 이용

dqn = DQN()
dqn.eval_net.load_state_dict(torch.load('DQN_Cartpole.pth'))


action_sequence_list = []
start = []
end = []

def main():
    for i_episode in range(1):
        total_reward = 0
        state, current_frame = env.reset()
        
        aa = 0
        action2 = 0
        for t in range(700):
                        
            #action = dqn.select_action(state)          
            
            next_state, reward, done, info, action2 = env.step(action2)
            action_sequence_list.append(action2)
            total_reward += reward

            if done: # 초기 데이터가 모이지 않았을 때
                print("total_reward",total_reward)
                print("end")
                start.append(current_frame)
                end.append(current_frame+t)
                print("current_frame: ",current_frame, "end_frame: ", current_frame+t)
                break
            print("action: ",action2 ,"reward: ", reward)
            env.render()
            state = next_state            
    
if __name__ == '__main__':
    main()

In [ ]:
start, end
initial_viewport_x =  21
initial_viewport_y =  25
action_dic = {0: ['-','-'], 1: ['-','0'],2: ['-','+'],3: ['0','-'],4: ['0', '0'],
            5: ['0','+'],6: ['+','-'],7: ['+','0'],8: ['+','+']}
action_list1 = []
# print(start[0], end[0])
# print(end[0]- start[0])
# print(len(action_sequence_list))
# print(action_sequence_list)
# print(len(action_sequence_list))
# print(action_sequence_list)

In [ ]:
#viewport_x = int(predictions[start[0]][0]['boxes'][0][0])
#viewport_y = int(predictions[start[0]][0]['boxes'][0][1])

In [ ]:
pixel_speed = 1
action_list1 = [initial_viewport_x, initial_viewport_y]

for action in action_sequence_list:
        
        if action_dic[action][0] == "-":            
            action_list1.append(viewport_y-pixel_speed)
#            action_list1.append(viewport_y+12-pixel_speed)
        elif action_dic[action][0] == "0":
            action_list1.append(viewport_y)
#            action_list1.append(viewport_y+12)
        elif action_dic[action][0] == "+":
            action_list1.append(viewport_y+pixel_speed)
#            action_list1.append(viewport_y+12+pixel_speed)

        if action_dic[action][1] == "-":
            action_list1.append(viewport_x-pixel_speed)
#            action_list1.append(viewport_x+20-pixel_speed)
        elif action_dic[action][1] == "0":
            action_list1.append(viewport_x)
#            action_list1.append(viewport_x+20)
        elif action_dic[action][1] == "+":
            action_list1.append(viewport_x+pixel_speed)
#            action_list1.append(viewport_x+20+pixel_speed)
        viewport_y, viewport_x = action_list1[-2:]
        #print(action_list1)

print(action_list1)

In [ ]:
%matplotlib notebook
%matplotlib notebook

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import patches as patches

fig, ax = plt.subplots()
SCREEN_WIDTH, SCREEN_HEIGHT = 20, 12

ims = []

# start, end = start[0], end[0]
# for i in range(start, end , 1):

for i in range(start[0], start[0]+len(action_sequence_list) , 1):
    acac1 = test_img_array[i][:1].sum(axis=0).detach().numpy()    
    acac1 = np.where(acac1 >= 0.1, 0.12, 0.0)
    acac2 = test_img_array[i][1:2].sum(axis=0).detach().numpy()    
    acac2 = np.where(acac2 >= 0.1, 0.24, 0.0)
    acac3 = test_img_array[i][2:3].sum(axis=0).detach().numpy()    
    acac3 = np.where(acac3 >= 0.1, 0.36, 0.0)
    acac4 = test_img_array[i][3:4].sum(axis=0).detach().numpy()    
    acac4 = np.where(acac4 >= 0.1, 0.48, 0.0)

    acac5 = test_img_array[i][4:5].sum(axis=0).detach().numpy()    
    acac5 = np.where(acac5 >= 0.1, 0.6, 0.0)
    acac6 = test_img_array[i][5:6].sum(axis=0).detach().numpy()    
    acac6 = np.where(acac6 >= 0.1, 0.72, 0.0)
    acac7 = test_img_array[i][6:7].sum(axis=0).detach().numpy()    
    acac7 = np.where(acac7 >= 0.1, 0.84, 0.0)
    acac8 = test_img_array[i][7:8].sum(axis=0).detach().numpy()    
    acac8 = np.where(acac8 >= 0.1, 0.96, 0.0)

    acac = acac1 + acac2 + acac3  + acac4  + acac5  + acac6
    acac = np.where(acac >= 1, 0.96, acac)
    
    print(i)
    viewport_y = int(action_list1[(i-start[0])*2+0])
    viewport_x = int(action_list1[(i-start[0])*2+1])
    print(viewport_y, viewport_x)
    
    #viewport_x = int(predictions[i][0]['boxes'][0][0])
    #viewport_y = int(predictions[i][0]['boxes'][0][1])
    
    acac[viewport_y:viewport_y+1, viewport_x:viewport_x+20]=1
    acac[viewport_y+12:viewport_y+13, viewport_x:viewport_x+21]=1    
    acac[viewport_y:viewport_y+12, viewport_x:viewport_x+1]=1
    acac[viewport_y:viewport_y+12, viewport_x+20:viewport_x+21]=1        
        
    im = ax.imshow(acac, cmap="RdYlBu_r")
    ttl = plt.text(0.5, 1.01, i, horizontalalignment='center', verticalalignment='bottom', transform=ax.transAxes)
    txt = plt.text(i,i,i)
    ims.append([im, ttl, txt])

ani = animation.ArtistAnimation(fig, ims, interval=500)
ani.save('Oserver_'+str(start)+'_'+str(end)+'_.gif', writer='imagemagick', fps=10)